In [0]:
import glob
import random
import pretty_midi
import IPython
import numpy as np
from tqdm import tnrange, tqdm_notebook, tqdm
from random import shuffle, seed
import numpy as np
import numpy as np
from numpy.random import choice
import pickle
import matplotlib.pyplot as plt

import unicodedata
import re
import numpy as np
import os
import io
import time

In [0]:
def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [0]:
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v1.0.0/maestro-v1.0.0-midi.zip

--2019-06-03 10:30:53--  https://storage.googleapis.com/magentadata/datasets/maestro/v1.0.0/maestro-v1.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.128, 2404:6800:4008:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46579421 (44M) [application/zip]
Saving to: ‘maestro-v1.0.0-midi.zip’

maestro-v1.0.0-midi 100%[===================>]  44.42M  28.5MB/s    in 1.6s    

2019-06-03 10:31:00 (28.5 MB/s) - ‘maestro-v1.0.0-midi.zip’ saved [46579421/46579421]



In [0]:
!unzip maestro-v1.0.0-midi.zip

Archive:  maestro-v1.0.0-midi.zip
   creating: maestro-v1.0.0/
  inflating: maestro-v1.0.0/LICENSE  
  inflating: maestro-v1.0.0/maestro-v1.0.0.csv  
   creating: maestro-v1.0.0/2017/
  inflating: maestro-v1.0.0/2017/MIDI-Unprocessed_045_PIANO045_MID--AUDIO-split_07-06-17_Piano-e_2-01_wav--2.midi  
  inflating: maestro-v1.0.0/2017/MIDI-Unprocessed_059_PIANO059_MID--AUDIO-split_07-07-17_Piano-e_2-03_wav--3.midi  
  inflating: maestro-v1.0.0/2017/MIDI-Unprocessed_046_PIANO046_MID--AUDIO-split_07-06-17_Piano-e_2-02_wav--1.midi  
  inflating: maestro-v1.0.0/2017/MIDI-Unprocessed_061_PIANO061_MID--AUDIO-split_07-07-17_Piano-e_2-05_wav--3.midi  
  inflating: maestro-v1.0.0/2017/MIDI-Unprocessed_083_PIANO083_MID--AUDIO-split_07-09-17_Piano-e_2_-06_wav--5.midi  
  inflating: maestro-v1.0.0/2017/MIDI-Unprocessed_083_PIANO083_MID--AUDIO-split_07-09-17_Piano-e_2_-06_wav--3.midi  
  inflating: maestro-v1.0.0/2017/MIDI-Unprocessed_049_PIANO049_MID--AUDIO-split_07-06-17_Piano-e_2-06_wav--5.midi  
  

# Get All Midi Files

In [0]:
def get_list_midi(folder = 'maestro-v1.0.0/**/*.midi', seed_int = 666):
    """Get the list of all midi file in the folders

    Parameters
    ==========
    folder : str
    The midi folder.
    seed_int : int
    the random seed.

    Returns
    =======
    The midi files

    """
    list_all_midi = glob.glob(folder)
    seed(seed_int)
    shuffle(list_all_midi)
    return list_all_midi

list_all_midi = get_list_midi()

In [0]:
class NoteTokenizer:
    
    def __init__(self):
        self.notes_to_index = {}
        self.index_to_notes = {}
        self.num_of_word = 0
        self.unique_word = 0
        self.notes_freq = {}
        
    def transform(self,list_array):
        """ Transform a list of note in string into index.

        Parameters
        ==========
        list_array : list
        list of note in string format

        Returns
        =======
        The transformed list in numpy array.

        """
        transformed_list = []
        for instance in list_array:
            transformed_list.append([self.notes_to_index[note] for note in instance])
        return np.array(transformed_list, dtype=np.int32)
 
    def partial_fit(self, notes):
        """ Partial fit on the dictionary of the tokenizer
        
        Parameters
        ==========
        notes : list of notes
        
        """
        for note in notes:
            note_str = ','.join(str(a) for a in note)
            if note_str in self.notes_freq:
                self.notes_freq[note_str] += 1
                self.num_of_word += 1
            else:
                self.notes_freq[note_str] = 1
                self.unique_word += 1
                self.num_of_word += 1
                self.notes_to_index[note_str], self.index_to_notes[self.unique_word] = self.unique_word, note_str
            
    def add_new_note(self, note):
        """ Add a new note into the dictionary

        Parameters
        ==========
        note : str
          a new note who is not in dictionary.  

        """
        assert note not in self.notes_to_index
        self.unique_word += 1
        self.notes_to_index[note], self.index_to_notes[self.unique_word] = self.unique_word, note
        
def generate_batch_song(list_all_midi, batch_music=16, start_index=0, fs=30, seq_len=50, use_tqdm=False):
    """
    Generate Batch music that will be used to be input and output of the neural network
    
    Parameters
    ==========
    list_all_midi : list
      List of midi files
    batch_music : int
      A number of music in one batch
    start_index : int
      The start index to be batched in list_all_midi
    fs : int
      Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    seq_len : int
      The sequence length of the music to be input of neural network
    use_tqdm : bool
      Whether to use tqdm or not in the function
    
    Returns
    =======
    Tuple of input and target neural network
    
    """
    
    assert len(list_all_midi) >= batch_music
    dict_time_notes = generate_dict_time_notes(list_all_midi, batch_music, start_index, fs, use_tqdm=use_tqdm)
    
    list_musics = process_notes_in_song(dict_time_notes, seq_len)
    collected_list_input, collected_list_target = [], []
     
    for music in list_musics:
        list_training, list_target = generate_input_and_target(music, seq_len)
        collected_list_input += list_training
        collected_list_target += list_target
    return collected_list_input, collected_list_target

def generate_dict_time_notes(list_all_midi, batch_song = 16, start_index=0, fs=30, use_tqdm=True):
    """ Generate map (dictionary) of music ( in index ) to piano_roll (in np.array)

    Parameters
    ==========
    list_all_midi : list
        List of midi files
    batch_music : int
      A number of music in one batch
    start_index : int
      The start index to be batched in list_all_midi
    fs : int
      Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    use_tqdm : bool
      Whether to use tqdm or not in the function

    Returns
    =======
    dictionary of music to piano_roll (in np.array)

    """
    assert len(list_all_midi) >= batch_song
    
    dict_time_notes = {}
    process_tqdm_midi = tqdm_notebook(range(start_index, min(start_index + batch_song, len(list_all_midi)))) if use_tqdm else range(start_index,  min(start_index + batch_song, len(list_all_midi)))
    for i in process_tqdm_midi:
        midi_file_name = list_all_midi[i]
        if use_tqdm:
            process_tqdm_midi.set_description("Processing {}".format(midi_file_name))
        try: # Handle exception on malformat MIDI files
            midi_pretty_format = pretty_midi.PrettyMIDI(midi_file_name)
            piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
            piano_roll = piano_midi.get_piano_roll(fs=fs)
            dict_time_notes[i] = piano_roll
        except Exception as e:
            print(e)
            print("broken file : {}".format(midi_file_name))
            pass
    return dict_time_notes

def generate_input_and_target(dict_keys_time, seq_len=50):
    """ Generate input and the target of our deep learning for one music.
    
    Parameters
    ==========
    dict_keys_time : dict
      Dictionary of timestep and notes
    seq_len : int
      The length of the sequence
      
    Returns
    =======
    Tuple of list of input and list of target of neural network.
    
       
    """
    # Get the start time and end time
    start_time, end_time = list(dict_keys_time.keys())[0], list(dict_keys_time.keys())[-1]
    list_training, list_target = [], []
    for index_enum, time in enumerate(range(start_time, end_time)):
        list_append_training, list_append_target = [], []
        start_iterate = 0
        flag_target_append = False # flag to append the test list
        if index_enum < seq_len:
            start_iterate = seq_len - index_enum - 1
            for i in range(start_iterate): # add 'e' to the seq list. 
                list_append_training.append('e')
                flag_target_append = True

        for i in range(start_iterate,seq_len):
            index_enum = time - (seq_len - i - 1)
            if index_enum in dict_keys_time:
                list_append_training.append(','.join(str(x) for x in dict_keys_time[index_enum]))      
            else:
                list_append_training.append('e')

        # add time + 1 to the list_append_target
        if time+1 in dict_keys_time:
            list_append_target.append(','.join(str(x) for x in dict_keys_time[time+1]))
        else:
            list_append_target.append('e')
        list_training.append(list_append_training)
        list_target.append(list_append_target)
    return list_training, list_target

def process_notes_in_song(dict_time_notes, seq_len = 50):
    """
    Iterate the dict of piano rolls into dictionary of timesteps and note played
    
    Parameters
    ==========
    dict_time_notes : dict
      dict contains index of music ( in index ) to piano_roll (in np.array)
    seq_len : int
      Length of the sequence
      
    Returns
    =======
    Dict of timesteps and note played
    """
    list_of_dict_keys_time = []
    
    for key in dict_time_notes:
        sample = dict_time_notes[key]
        times = np.unique(np.where(sample > 0)[1])
        index = np.where(sample > 0)
        dict_keys_time = {}

        for time in times:
            index_where = np.where(index[1] == time)
            notes = index[0][index_where]
            dict_keys_time[time] = notes
        list_of_dict_keys_time.append(dict_keys_time)
    return list_of_dict_keys_time



In [0]:
sampled_200_midi = list_all_midi[0:500]

In [0]:
batch = 1
start_index = 0
note_tokenizer = NoteTokenizer()

for i in tqdm_notebook(range(len(sampled_200_midi))):
    dict_time_notes = generate_dict_time_notes(sampled_200_midi, batch_song=1, start_index=i, use_tqdm=False, fs=20)
    full_notes = process_notes_in_song(dict_time_notes)
    for note in full_notes:
        note_tokenizer.partial_fit(list(note.values()))

In [0]:
dict_time_notes = generate_dict_time_notes(sampled_200_midi, batch_song=1, start_index=0, use_tqdm=False, fs=5)

In [0]:
times = np.unique(np.where(dict_time_notes[0] > 0)[1])
index = np.where(dict_time_notes[0] > 0)

In [0]:
index

(array([ 24,  24,  24, ..., 103, 105, 105]),
 array([1103, 1198, 1225, ..., 6275, 3229, 4965]))

In [0]:
process_notes_in_song(dict_time_notes)

[{7: array([69]),
  8: array([59, 71]),
  12: array([52, 64]),
  13: array([52, 54, 66]),
  17: array([69]),
  18: array([57, 69]),
  19: array([88]),
  20: array([64, 76]),
  21: array([64, 76]),
  22: array([76]),
  27: array([69]),
  28: array([59, 71, 72]),
  30: array([64]),
  32: array([64]),
  33: array([54, 66]),
  35: array([57, 69]),
  37: array([69]),
  38: array([71]),
  43: array([52, 64]),
  45: array([57, 93]),
  47: array([69, 81]),
  48: array([70, 82]),
  50: array([62, 77, 81]),
  53: array([65, 77]),
  55: array([62, 79, 86]),
  58: array([67, 81]),
  62: array([72]),
  63: array([62, 74, 76]),
  65: array([65]),
  67: array([71]),
  68: array([60, 72]),
  70: array([64, 76]),
  73: array([59, 72]),
  75: array([53, 65]),
  76: array([53, 57, 65]),
  77: array([64]),
  78: array([76]),
  79: array([59, 71]),
  80: array([57, 69]),
  81: array([57, 69]),
  82: array([57, 72]),
  85: array([57, 64, 88, 93]),
  87: array([69]),
  88: array([70, 82]),
  90: array([62, 7

In [0]:
note_tokenizer.add_new_note('e') # Add empty notes

In [0]:
unique_notes = note_tokenizer.unique_word
print(unique_notes)

181775


In [0]:
seq_len = 50
EPOCHS = 4
BATCH_SONG = 16
BATCH_NNET_SIZE = 96
TOTAL_SONGS = len(sampled_200_midi)
FRAME_PER_SECOND = 20

In [0]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
class AutoRegression(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=2, output_emb=128, bidirectional=False):
        """ 
        A simple sequential encoder for titles.
        x -> emb -> conv -> global_max -> relu -> dense
        """
        super(self.__class__, self).__init__()
       

        self.logits_fc = nn.Linear(input_size * output_size, output_size)
        
        self.softmax = nn.Softmax(dim = 1)
        

    def forward(self, input_sequences, hidden=None):
        logits = self.logits_fc(input_sequences.view(input_sequences.shape[0], -1))
                
        out = self.softmax(logits)

        
        return out

In [0]:
val_set = list_all_midi[0:1]

In [0]:
val_inputs_nnet_large, val_outputs_nnet_large = generate_batch_song(val_set, fs=20, batch_music=1)
val_inputs_nnet_large = np.array(note_tokenizer.transform(val_inputs_nnet_large), dtype=np.int32)
val_outputs_nnet_large = np.array(note_tokenizer.transform(val_outputs_nnet_large), dtype=np.int32)
val_index = np.arange(start=0, stop=len(val_inputs_nnet_large))
val_inputs, val_outputs = val_inputs_nnet_large[val_index], val_outputs_nnet_large[val_index]
encoded_inputs = []
for inpt in val_inputs:
    sample = []
    for label in inpt:
        note_ids = note_tokenizer.index_to_notes[label]
        encoded_input = np.zeros(128)
        for note_id in note_ids.split(','):
            if note_id != 'e':
                encoded_input[int(note_id)] = 1
        sample.append(encoded_input)
    encoded_inputs.append(sample)

#                     print(encoded_inputs[0])
val_inputs_tensor = torch.tensor(encoded_inputs, dtype=torch.float).cuda()
#                     outputs_tensor = torch.tensor(outputs, dtype=torch.float32)
encoded_outputs = []
for output in val_outputs:
    note_ids = note_tokenizer.index_to_notes[output[0]]
    encoded_output = np.zeros(128)
    for note_id in note_ids.split(','):
        if note_id != 'e':
            encoded_output[int(note_id)] = 1
    encoded_outputs.append(encoded_output)

#                     outputs_tensor = torch.tensor(outputs, dtype=torch.long).view(-1)
val_outputs_tensor = torch.tensor(encoded_outputs, dtype=torch.float).cuda()

In [0]:
from IPython import display

class TrainModel:
  
    def __init__(self, epochs, note_tokenizer, sampled_200_midi, frame_per_second, 
               batch_nnet_size, batch_song, optimizer, loss_fn, total_songs, model, seq_len):
        self.epochs = epochs
        self.note_tokenizer = note_tokenizer
        self.sampled_200_midi = sampled_200_midi
        self.frame_per_second = frame_per_second
        self.batch_nnet_size = batch_nnet_size
        self.batch_song = batch_song
        self.optimizer = optimizer
        #     self.checkpoint = checkpoint
        self.loss_fn = loss_fn
        #     self.checkpoint_prefix = checkpoint_prefix
        self.total_songs = total_songs
        self.model = model
        self.seq_len = seq_len
        self.train_loss_history = []
        self.val_loss_history = []
    
    def train(self):
        loss_history = []
      
      
        for epoch in tqdm_notebook(range(self.epochs),desc='epochs'):
            # for each epochs, we shufle the list of all the datasets
            shuffle(self.sampled_200_midi)
            loss_total = 0
            steps = 0
            steps_nnet = 0

            # We will iterate all songs by self.song_size
            for i in tqdm_notebook(range(0,self.total_songs, self.batch_song), desc='MUSIC'):

                steps += 1
                inputs_nnet_large, outputs_nnet_large = generate_batch_song(
                    self.sampled_200_midi, self.batch_song, start_index=i, fs=self.frame_per_second, 
                    seq_len=seq_len, use_tqdm=False) # We use the function that have been defined here
                inputs_nnet_large = np.array(self.note_tokenizer.transform(inputs_nnet_large), dtype=np.int32)
                outputs_nnet_large = np.array(self.note_tokenizer.transform(outputs_nnet_large), dtype=np.int32)

                index_shuffled = np.arange(start=0, stop=len(inputs_nnet_large))
                np.random.shuffle(index_shuffled)

#                 for nnet_steps in tqdm_notebook(range(0,len(index_shuffled),self.batch_nnet_size)):
                for nnet_steps in tqdm_notebook(range(0,len(index_shuffled),self.batch_nnet_size)):
                    steps_nnet += 1
                    current_index = index_shuffled[nnet_steps:nnet_steps+self.batch_nnet_size]
                    inputs, outputs = inputs_nnet_large[current_index], outputs_nnet_large[current_index]
                    
                    
                    encoded_inputs = []
                    for inpt in inputs:
                        sample = []
                        for label in inpt:
                            note_ids = note_tokenizer.index_to_notes[label]
                            encoded_input = np.zeros(128)
                            for note_id in note_ids.split(','):
                                if note_id != 'e':
                                    encoded_input[int(note_id)] = 1
                            sample.append(encoded_input)
                        encoded_inputs.append(sample)
                    
#                     print(encoded_inputs[0])
                    inputs_tensor = torch.tensor(encoded_inputs, dtype=torch.float).cuda()
#                     outputs_tensor = torch.tensor(outputs, dtype=torch.float32)
                    encoded_outputs = []
                    for output in outputs:
                        note_ids = note_tokenizer.index_to_notes[output[0]]
                        encoded_output = np.zeros(128)
                        for note_id in note_ids.split(','):
                            if note_id != 'e':
                                encoded_output[int(note_id)] = 1
                        encoded_outputs.append(encoded_output)
                        
#                     outputs_tensor = torch.tensor(outputs, dtype=torch.long).view(-1)
                    outputs_tensor = torch.tensor(encoded_outputs, dtype=torch.float).cuda()

                    # To make sure no exception thrown by tensorflow on autograph
                    if len(inputs) // self.batch_nnet_size != 1:
                        break
                    self.model.train()
                    prediction, _ = self.model.forward(inputs_tensor)#, self.seq_len)
                    loss = self.loss_fn(prediction, outputs_tensor)
                    loss.backward()
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    loss_total += loss
                    n_verbose = 20
                    if steps_nnet % n_verbose == 0:
                        loss_mean = (loss_total / n_verbose).detach().cpu().numpy()
                        print("epochs {} | Steps {} | total loss : {}".format(epoch + 1, steps_nnet, loss_mean))
                        self.train_loss_history.append(loss_mean)
                        loss_total = 0
                        
                        val_prediction, _ = self.model.forward(val_inputs_tensor)#, self.seq_len)
                        val_loss = self.loss_fn(val_prediction, val_outputs_tensor).detach().cpu().numpy()
                        print(f'val loss: {val_loss}')
                        self.val_loss_history.append(val_loss)
                        
                    self.model.eval()

#         checkpoint.save(file_prefix = self.checkpoint_prefix)

In [0]:
learning_rate = 0.001
seq_len = 50
EPOCHS = 2
BATCH_SONG = 16
BATCH_NNET_SIZE = 512
TOTAL_SONGS = len(sampled_200_midi)
FRAME_PER_SECOND = 20

def compute_loss(prediction, targets):
    loss_table = -torch.log(prediction) * targets - torch.log(1 - prediction) * (1 - targets)
    loss = loss_table.mean()
    return loss

In [0]:
ar = AutoRegression(input_size=seq_len, hidden_size=16, output_size=128, bidirectional=False).cuda()

optimizer = torch.optim.Adam(ar.parameters(), lr=learning_rate)
train_class = TrainModel(EPOCHS, note_tokenizer, sampled_200_midi, FRAME_PER_SECOND,
                BATCH_NNET_SIZE, BATCH_SONG, optimizer, compute_loss, TOTAL_SONGS, ar, seq_len)
train_class.train()

epochs 1 | Steps 20 | total loss : 0.07374808937311172
val loss: 0.07047370076179504
epochs 1 | Steps 40 | total loss : 0.06184237077832222
val loss: 0.06615887582302094
epochs 1 | Steps 60 | total loss : 0.057760417461395264
val loss: 0.06394398957490921
epochs 1 | Steps 80 | total loss : 0.0551689937710762
val loss: 0.062475625425577164
epochs 1 | Steps 100 | total loss : 0.05317066237330437
val loss: 0.06135101988911629
epochs 1 | Steps 120 | total loss : 0.0522085539996624
val loss: 0.060456134378910065
epochs 1 | Steps 140 | total loss : 0.0508100651204586
val loss: 0.05979150906205177
epochs 1 | Steps 160 | total loss : 0.05032694339752197
val loss: 0.05928945541381836
epochs 1 | Steps 180 | total loss : 0.04929129034280777
val loss: 0.05884183943271637
epochs 1 | Steps 200 | total loss : 0.04848378524184227
val loss: 0.05841228738427162
epochs 1 | Steps 220 | total loss : 0.04727441072463989
val loss: 0.058009903877973557
epochs 1 | Steps 240 | total loss : 0.047599803656339645


epochs 1 | Steps 260 | total loss : 0.04617990180850029
val loss: 0.057111166417598724
epochs 1 | Steps 280 | total loss : 0.05136873200535774
val loss: 0.056172244250774384
epochs 1 | Steps 300 | total loss : 0.05017605423927307
val loss: 0.05554298311471939
epochs 1 | Steps 320 | total loss : 0.048948295414447784
val loss: 0.05523731932044029
epochs 1 | Steps 340 | total loss : 0.0476575642824173
val loss: 0.05488517880439758
epochs 1 | Steps 360 | total loss : 0.047173209488391876
val loss: 0.054603688418865204
epochs 1 | Steps 380 | total loss : 0.04653980955481529
val loss: 0.05433816835284233
epochs 1 | Steps 400 | total loss : 0.04600564017891884
val loss: 0.05407541245222092
epochs 1 | Steps 420 | total loss : 0.0456802099943161
val loss: 0.05387602373957634
epochs 1 | Steps 440 | total loss : 0.04531031474471092
val loss: 0.05355115234851837
epochs 1 | Steps 460 | total loss : 0.044654008001089096
val loss: 0.05339694395661354
epochs 1 | Steps 480 | total loss : 0.044259052723

epochs 1 | Steps 600 | total loss : 0.043177489191293716
val loss: 0.05206843093037605
epochs 1 | Steps 620 | total loss : 0.04632321745157242
val loss: 0.05180405452847481
epochs 1 | Steps 640 | total loss : 0.045420434325933456
val loss: 0.051620397716760635
epochs 1 | Steps 660 | total loss : 0.04433411732316017
val loss: 0.05154148116707802
epochs 1 | Steps 680 | total loss : 0.04376547038555145
val loss: 0.051376648247241974
epochs 1 | Steps 700 | total loss : 0.04465194046497345
val loss: 0.05125761404633522
epochs 1 | Steps 720 | total loss : 0.04329864680767059
val loss: 0.0511280857026577
epochs 1 | Steps 740 | total loss : 0.042795490473508835
val loss: 0.05108373612165451
epochs 1 | Steps 760 | total loss : 0.04240618646144867
val loss: 0.05095115303993225
epochs 1 | Steps 780 | total loss : 0.043021876364946365
val loss: 0.05094349384307861
epochs 1 | Steps 800 | total loss : 0.04232596978545189
val loss: 0.05085121467709541
epochs 1 | Steps 820 | total loss : 0.04224575683

epochs 1 | Steps 960 | total loss : 0.03987153246998787
val loss: 0.050303831696510315
epochs 1 | Steps 980 | total loss : 0.04492824897170067
val loss: 0.04989032447338104
epochs 1 | Steps 1000 | total loss : 0.044373154640197754
val loss: 0.04954403638839722
epochs 1 | Steps 1020 | total loss : 0.04349477216601372
val loss: 0.049259305000305176
epochs 1 | Steps 1040 | total loss : 0.04320283234119415
val loss: 0.04907603934407234
epochs 1 | Steps 1060 | total loss : 0.04312794283032417
val loss: 0.04894159361720085
epochs 1 | Steps 1080 | total loss : 0.04355086758732796
val loss: 0.04877233877778053
epochs 1 | Steps 1100 | total loss : 0.04224487766623497
val loss: 0.04862324893474579
epochs 1 | Steps 1120 | total loss : 0.042105745524168015
val loss: 0.048578206449747086
epochs 1 | Steps 1140 | total loss : 0.0422140508890152
val loss: 0.04853622242808342
epochs 1 | Steps 1160 | total loss : 0.04223774001002312
val loss: 0.048445865511894226
epochs 1 | Steps 1180 | total loss : 0.0

epochs 1 | Steps 1340 | total loss : 0.0411190390586853
val loss: 0.04777616634964943
epochs 1 | Steps 1360 | total loss : 0.04385443031787872
val loss: 0.04756592586636543
epochs 1 | Steps 1380 | total loss : 0.0429212860763073
val loss: 0.04738326370716095
epochs 1 | Steps 1400 | total loss : 0.043101079761981964
val loss: 0.047295231372117996
epochs 1 | Steps 1420 | total loss : 0.04176545515656471
val loss: 0.047201499342918396
epochs 1 | Steps 1440 | total loss : 0.04200730845332146
val loss: 0.04713630676269531
epochs 1 | Steps 1460 | total loss : 0.04191678762435913
val loss: 0.04716767743229866
epochs 1 | Steps 1480 | total loss : 0.041745755821466446
val loss: 0.04707040265202522
epochs 1 | Steps 1500 | total loss : 0.04133889079093933
val loss: 0.04704895615577698
epochs 1 | Steps 1520 | total loss : 0.04152883216738701
val loss: 0.04700952395796776
epochs 1 | Steps 1540 | total loss : 0.040788162499666214
val loss: 0.046998750418424606
epochs 1 | Steps 1560 | total loss : 0.

epochs 1 | Steps 1600 | total loss : 0.042219825088977814
val loss: 0.046501655131578445
epochs 1 | Steps 1620 | total loss : 0.04434681683778763
val loss: 0.04625605419278145
epochs 1 | Steps 1640 | total loss : 0.04370144382119179
val loss: 0.04604426771402359
epochs 1 | Steps 1660 | total loss : 0.04294996336102486
val loss: 0.04598407447338104
epochs 1 | Steps 1680 | total loss : 0.04323580116033554
val loss: 0.04585152864456177
epochs 1 | Steps 1700 | total loss : 0.042821962386369705
val loss: 0.04585505276918411
epochs 1 | Steps 1720 | total loss : 0.04255985841155052
val loss: 0.045786451548337936
epochs 1 | Steps 1740 | total loss : 0.04270709678530693
val loss: 0.0457245409488678
epochs 1 | Steps 1760 | total loss : 0.04238013178110123
val loss: 0.04573237895965576
epochs 1 | Steps 1780 | total loss : 0.04227903485298157
val loss: 0.045690447092056274
epochs 1 | Steps 1800 | total loss : 0.042093630880117416
val loss: 0.045602574944496155
epochs 1 | Steps 1820 | total loss : 

epochs 1 | Steps 1940 | total loss : 0.039801109582185745
val loss: 0.04543140530586243
epochs 1 | Steps 1960 | total loss : 0.041301120072603226
val loss: 0.045410193502902985
epochs 1 | Steps 1980 | total loss : 0.04065212979912758
val loss: 0.045453429222106934
epochs 1 | Steps 2000 | total loss : 0.0401526503264904
val loss: 0.04536629095673561
epochs 1 | Steps 2020 | total loss : 0.03995935246348381
val loss: 0.04539493843913078
epochs 1 | Steps 2040 | total loss : 0.03984782472252846
val loss: 0.04536569491028786
epochs 1 | Steps 2060 | total loss : 0.039701640605926514
val loss: 0.045338939875364304
epochs 1 | Steps 2080 | total loss : 0.03957171365618706
val loss: 0.045309845358133316
epochs 1 | Steps 2100 | total loss : 0.03962786868214607
val loss: 0.045321784913539886
epochs 1 | Steps 2120 | total loss : 0.03880865499377251
val loss: 0.04525916278362274
epochs 1 | Steps 2140 | total loss : 0.03961018845438957
val loss: 0.04526384919881821
epochs 1 | Steps 2160 | total loss :

KeyboardInterrupt: ignored

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform([list(range(128))])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [0]:
def generate_from_random(unique_notes, batch_size, seq_len=50):
# #     random_notes = torch.randint(high=unique_notes, size=(batch_size, seq_len), dtype=torch.long)#.cuda()
#     random_notes = torch.randint(high=unique_notes, size=(batch_size, seq_len), dtype=torch.long).numpy()
#     random_notes = mlb.transform([random_notes])
    
    random_notes = [[x] for x in np.random.choice(128, 50)]
    random_notes = mlb.transform(random_notes)
    return torch.LongTensor(random_notes).view(1, random_notes.shape[0], random_notes.shape[1])

def generate_from_constant(unique_notes, batch_size, seq_len=50, note=64):
# #     random_notes = torch.randint(high=unique_notes, size=(batch_size, seq_len), dtype=torch.long)#.cuda()
#     random_notes = torch.randint(high=unique_notes, size=(batch_size, seq_len), dtype=torch.long).numpy()
#     random_notes = mlb.transform([random_notes])
    
    random_notes = [[seq_len] for _ in range(seq_len)]
    random_notes = mlb.transform(random_notes)
    return torch.LongTensor(random_notes).view(1, random_notes.shape[0], random_notes.shape[1])
  
def generate_notes(previous_notes, model, batch_size, unique_notes, max_generated=1000, seq_len=50, threshold=0.015):
#     previous_notes = torch.LongTensor(previous_notes)
    for n_step in range(max_generated - seq_len):
        prev_notes_truncated = previous_notes[:, -seq_len:, :].float()
#         print(prev_notes_truncated)
        next_notes_probs, _ = model(prev_notes_truncated.cuda())
#         new_notes = torch.arange(128)[next_notes_probs[0] > threshold]
        new_notes = torch.multinomial(next_notes_probs, 2)
#         print(new_notes)
        new_notes = list(new_notes.cpu().numpy()[0])
#         print(new_notes)
        
#         print(new_notes)
        new_note_vector = mlb.transform([new_notes])  # WRONG INDEXES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#         print(new_note_vector)
        new_note_vector = torch.LongTensor(new_note_vector).view(1, new_note_vector.shape[0], new_note_vector.shape[1])
#         for note, note_prob in enumerate(next_notes_probs):
#             if np.random.uniform() < note_prob:
#                 next_notes.append(note)
        
#         NEW_NOTE = torch.multinomial(next_notes_probs, 1)
# #         print(NEW_NOTE)
# #         print(previous_notes)
#         previous_notes = torch.cat([previous_notes[:, :], NEW_NOTE], dim=1, out=previous_notes)
        previous_notes = torch.cat([previous_notes[:, :, :], new_note_vector], dim=1)
        
    
    return previous_notes

def write_midi_file_from_generated(generate, midi_file_name = "result.mid", start_index=49, fs=20, max_generated=1000):
    array_piano_roll = np.zeros((128, max_generated), dtype=np.int16)
    
    for index, notes in enumerate(generate[:, :]):
#         print(notes)
        nonzero_notes = np.nonzero(notes)
        if len(nonzero_notes) > 0:
#             print(nonzero_notes)
            nonzero_notes = nonzero_notes[0]
        else:
            continue

        for j in nonzero_notes:
            print(f'j = {j}, index = {index}')
            array_piano_roll[int(j),index] = 1
#     print(array_piano_roll.sum())
    generate_to_midi = piano_roll_to_pretty_midi(array_piano_roll, fs=fs)
    for note in generate_to_midi.instruments[0].notes:
        note.velocity = 100
    generate_to_midi.write(midi_file_name)
    
    return array_piano_roll

In [0]:
ar.eval()
max_generate = 1000
unique_notes = note_tokenizer.unique_word
seq_len=50
previous_notes = generate_from_constant(128, 1, note=100)
previous_notes = generate_notes(previous_notes, ar, 1, 128, max_generated=max_generate, threshold=0.001)
array_piano_roll = write_midi_file_from_generated(previous_notes[0], "ar.mid", start_index=seq_len-1, 
                                                  fs=10, max_generated = max_generate)

j = 50, index = 0
j = 50, index = 1
j = 50, index = 2
j = 50, index = 3
j = 50, index = 4
j = 50, index = 5
j = 50, index = 6
j = 50, index = 7
j = 50, index = 8
j = 50, index = 9
j = 50, index = 10
j = 50, index = 11
j = 50, index = 12
j = 50, index = 13
j = 50, index = 14
j = 50, index = 15
j = 50, index = 16
j = 50, index = 17
j = 50, index = 18
j = 50, index = 19
j = 50, index = 20
j = 50, index = 21
j = 50, index = 22
j = 50, index = 23
j = 50, index = 24
j = 50, index = 25
j = 50, index = 26
j = 50, index = 27
j = 50, index = 28
j = 50, index = 29
j = 50, index = 30
j = 50, index = 31
j = 50, index = 32
j = 50, index = 33
j = 50, index = 34
j = 50, index = 35
j = 50, index = 36
j = 50, index = 37
j = 50, index = 38
j = 50, index = 39
j = 50, index = 40
j = 50, index = 41
j = 50, index = 42
j = 50, index = 43
j = 50, index = 44
j = 50, index = 45
j = 50, index = 46
j = 50, index = 47
j = 50, index = 48
j = 50, index = 49
j = 49, index = 50
j = 50, index = 51
j = 49, index = 52
j =

In [0]:
def generate_from_random(unique_notes, seq_len=50):
    generate = np.random.randint(0,unique_notes,seq_len).tolist()
    return torch.LongTensor(generate)
    
def generate_from_one_note(note_tokenizer, new_notes='35'):
    generate = [note_tokenizer.notes_to_index['e'] for i in range(49)]
    generate += [note_tokenizer.notes_to_index[new_notes]]
    return torch.LongTensor(generate)

In [0]:
max_generate = 200
unique_notes = note_tokenizer.unique_word
seq_len=50
generate = generate_from_random(unique_notes, seq_len)
generate = generate_notes(generate, rnn, unique_notes, max_generate, seq_len)
write_midi_file_from_generated(generate, "random.mid", start_index=seq_len-1, fs=7, max_generated = max_generate)

[[3.1421590e-05 2.7522279e-05 2.5735475e-05 ... 2.6609121e-05
  3.3043350e-05 2.4890751e-05]]


RuntimeError: index out of range at /Users/administrator/nightlies/pytorch-1.0.0/wheel_build_dirs/wheel_3.6/pytorch/aten/src/TH/generic/THTensorEvenMoreMath.cpp:191

In [0]:
max_generate = 300
unique_notes = note_tokenizer.unique_word
seq_len=50
generate = generate_from_one_note(note_tokenizer, '72')
generate = generate_notes(generate, rnn, unique_notes, max_generate, seq_len)
write_midi_file_from_generated(generate, "one_note.mid", start_index=seq_len-1, fs=8, max_generated = max_generate)